In [1]:
import re
import requests
import pandas as pd
from hashlib import md5
import os

In [2]:
def get_proxy_gen(file='proxy.txt'):
    while True:
        with open(file, 'r') as f:
            for proxy in f:
                yield proxy.strip()

def get_proxy():
    global proxer
    proxy = next(proxer)
    return proxy

def get_html_page(url):
    while True:
        try:
            proxies = {
                'https': 'http://' + get_proxy(),
            }
            req = requests.get(url, proxies=proxies)
            if (not req.ok) or (req.text.find('Доступ временно заблокирован') > 0) or\
                    (req.text.find('Доступ с вашего IP-адреса временно ограничен') > 0):
                continue
            return req
        except Exception as e:
#             print(e)
            continue
proxer = get_proxy_gen()

In [3]:
urls = list(pd.read_csv("data_30.11_21-10.csv")['url'])
data = {'url':[], 'banned':[], 'comment':[]}
alerts = ['Ой! Такой страницы на нашем сайте нет :(', 'Страница не найдена', 'Пользователь заблокирован']

In [4]:
if os.path.isfile('checked_accounts.txt'):
    with open('checked_accounts.txt', 'r') as f:
        try:
            url = next(f)
            if url.find('avito.ru') > 0:
                while urls[0] != url:
                    urls.pop(0)
        except Exception as e:
            print('checked_accounts.txt error:', e)

In [5]:
k = 0
HASH_MOD = 0

for url in urls:
    if int(md5(url.encode('utf-8')).hexdigest(), base=16) % 3 != HASH_MOD:
        continue
    page = get_html_page(url)
    data['url'].append(url)
    k += 1
    for al in alerts:
        if page.text.find(al) > 0:
            print(url)
            data['banned'].append(1)
            data['comment'].append(al)
            break
    else:
        data['banned'].append(0)
        data['comment'].append("")
        
    if k % 30 == 0:
        df = pd.DataFrame(data)
        df.to_csv(os.path.join('banned', 'data{}'.format(str(datetime.now()).replace(':', '.'))))
        data = {'url':[], 'banned':[], 'comment':[]}
        with open('checked_accounts.txt', 'w') as f:
            print(url, file=f)
        
df = pd.DataFrame(data)
df.to_csv("banned.csv")

KeyboardInterrupt: 